## Paper

1. GAN에 대한 이론적 개념 제시: minimax problem
2. GAN이 풀어야 하는 problem이 global minimum에서 unique solution을 갖고 어떤 조건을 만족하면 해당 solution으로 수렴한다는 것을 증명

## GAN에 대한 개념적 소개

GAN은 크게 두 부분으로 구성되어 있다. <br />
1. Image를 만들어내는 Generator 
2. Generator에서 만들어진 이미지를 평가하는 Discriminator

이 두 구성 요소가 서로 대립(Adversarial)하며 서로의 성능을 점차 개선해 나간다.

### Ian Goodfellow의 예시

지폐위조범(Generator)은 경찰을 최대한 열심히 속이려고 하고 다른 한편에서는 경찰(Discriminator)이 이렇게 위조된 지폐를 진짜와 감별하려고(Classify) 노력한다. <br />
이런 경쟁 속에서 두 그룹 모두 속이고 구별하는 서로의 능력이 발전하게 되고 결과적으로는 진짜 지폐와 위조 지폐를 구별할 수 없을 정도(구별할 확률 $p_d$=0.5)에 이른다.

## GAN Model

Generative model $G$는 우리가 가지고 있는 data $x$의 distribution을 알아내려고 노력한다. <br />
만약 $G$가 정확히 data distribution을 모사할 수 있다면 거기에서 뽑은 sample은 완벽히 data와 구별할 수 없을 것이다. <br />
한편 discriminator model $D$에서는 현재 자기가 보고 있는 sample이 training data에서 온 것(진짜)인지 혹은 $G$로부터 만들어진 것(가짜)인지를 구별하여 <br />
각각의 경우에 대한 확률을 estimate한다.

<img src="./Images/1.png" width=600 />

위 그림을 보면 $D$의 입장에서는 data로 부터 뽑은 sample $x$는 $D(x)$=1이 되고, $G$에 임의의 noise distribution으로부터 뽑은 input $z$를 넣고 만들어진 <br /> 
sample에 대해서는 $D(G(z))$=0이 되도록 노력한다. <br />
즉, $D$는 실수할 확률을 낮추기(mini)위해 노력하고 반대로 $G$는 $D$가 실수할 확률을 높이기(max)위해 노력하는데, 이는 "minimax two-player game or <br /> 
minimax problem"이라 할 수 있다.

여기서 $G$와 $D$는 반드시 neural network로 만들 필요는 없으며, 어떤 model이든 이 역할을 서로 잘 해줄 수 만 있다면 상관이 없다. <br />
하지만 neural network를 사용한 것이 여러모로 실제 적용시에 장점이 있고, 그리고 결과도 좋기 때문에 결국 Generative Adversarial Nets가 되었다.

## Adversarial Nets

$G$와 $D$를 둘 다 multi-layer perceptrons model을 사용하면 <br />
Generator's distribution $p_g$ over data $x$를 학습하기 위해 generator의 input으로 들어갈 noise variable $p_z(z)$에 대한 prior를 정의하고, <br />
data space로의 mapping을 $G(z;{ \theta  }_{ g })$라 표현할 수 있다. <br />
여기서 G는 미분 가능한 함수로써 ${ \theta  }_{ g }$를 parameter로 갖는 mult-layer perceptron이다.

Discriminator 역시 multi-layer perceptron으로 $D(x;{ \theta  }_{ d })$로 나타내며 output은 single scalar 값이 된다.(확률이므로) <br />
$D(x)$는 $x$가 $p_g$가 아닌 data distribution으로부터 왔을 확률을 나타낸다.

이를 수식으로 정리하면 다음과 같은 value function $V(G,D)$에 대한 minimax problem을 푸는 것과 같아진다. <br />

<img src="./Images/2.png" width=700 />

가장 이상적인 상황에서의 $D$ 입장을 생각해보자. <br />
$D$가 보는 sample $x$가 실제로 data distribution으로부터 온 것이라면 $D(x)$=1 이므로 첫 번째 term에서 $log$ 값이 사라지고 $G(z)$가 만들어낸 것이라면 $D(G(z)$=0이므로 <br /> 
두 번째 term 역시 0으로 사라진다. <br />
이 때가 D의 입장에서 V의 최대값을 얻을 수 있다는 것은 자명하다.

반대로 $G$의 입장에서 생각해보아도 상황은 비슷하다.

<img src="./Images/3.png" width=600 />

위 그림에서 검은 점선이 data generating distribution이고, 파란 점선이 discriminator distribution, 녹색 선이 generative distribution이다. <br />
밑에 $x$와 $z$선은 각각 $x$와 $z$의 domain을 나타내며 위로 뻗은 화살표가 $x=G(z)$의 mapping을 보여준다. 

즉, 처음 시작할 때는 (a)와 같이 $p_g$가 $p_data$와 전혀 다르게 생긴 것을 볼 수 있고 이 상태에서 discriminator가 두 distribution을 구별하기 위해 <br />
학습을 하면 (b)와 같이 좀 더 smooth하고 잘 구별하는 distribution이 만들어진다. <br />
이후 $G$가 현재 discriminator가 구별하기 어려운 방향으로 학습을 하면 (c)와 같이 좀 더 $p_g$가 $p_data$와 더 가까워지게 되고, <br />
이런 식으로 학습을 반복하다보면 결국에는 $p_g$와 $p_data$가 되어 discriminator가 둘을 전혀 구별하지 못하는 $D(x)$=0.5의 상태가 된다.

논문에 나오는 Tip중 하나는, 위 value function에서 $log(1-D(G(z))$ 부분을 $G$에 대해 minimize하는 대신 $log(D(G(z))$를 maximize 하도록 $G$를 학습시킨다. <br />
이 부분은 이론적인 동기로부터 수정을 한 것이 아니라 순수하게 실용적인 측면에서 적용을 하게 된 것이다.

이렇게 하는 이유는, 학습 초기를 생각해 보면 $G$가 초기에는 아주 이상한 image들을 생성하기 때문에 $D$가 너무도 쉽게 이를 real image와 구별하게 되고 따라서 $log(1-D(G(z))$ 값이 매우 saturate하여 gradient를 계산해보면 아주 작은 값이 나오기 때문에 학습이 매우 느려지기 때문이다.

따라서 문제를 $G=argmax_G log(D(G(z))$로 바꾸게 되면, 초기에 $D$가 $G$로 나온 image를 잘 구별한다고 해도 위와 같은 문제가 생기지 않기 때문에 원래 문제와 같은 fixed point를 얻게 되면서도 stronger gradient를 줄 수 있는 상당히 좋은 방법이다.

## Theoretical Result

지금까지 사용해왔던 MLP를 사용하여 내용을 이어가는 것이 아니라 이론적 증명을 편하게 하기 위해 다른 방법을 사용한다. <br />
저자는 non-parametric setting을 사용하였다고 표현하는데, MLP를 사용하는 것은 결국 parameter $\theta$를 학습하는 것이므로 직접적으로 <br />
probability density function을 학습하는 것과는 차이가 있다는 것을 언급한다. <br />
즉, 앞으로 나올 이론적 증명들은 model이 infinite capacity를 가지고 있으며, 수렴에 대해서도 probability density function 공간에서 진행하고 있다. <br />
그러면 실제로 MLP를 사용할 때는 논문에서 하고자 하는 증명들이 정확히 적용되는 것은 아니다라는 것을 알 수 있다. <br />
하지만 실용적인 측면에서 실제로 학습을 시킬 때, MLP를 model로 사용하면 성능이 잘 나오기도 하고 MLP를 사용하면 직관적이고 단순하지만 강력한 backpropagation을 사용하는 것이 가능하기 때문에 inference를 위한 별도의 가정등이 필요하지 않다. <br />
다른 방법으로는 probability를 학습하기 위해 Markov chain Monte Carlo와 같은 방법들이 있지만 이는 효율적이지 않다. <br />
Markov chain Monte Carlo는 High dimensional vector space에서 특히 효율적이지 않은데 만약 이를 이용하여 Probability distribution을 찾는다고 생각해보면 점을 sampling하여 distribution을 유추하겠다는 것인데, 이는 거의 불가능에 가깝다

이론적 결과 부분에서는 minimax problem이 $p_g={p}_{data}$에서 global optimum을 갖는다는 것을 보이고 <br /> 
이어서 논문에서 소개하는 알고리즘이 global optimum을 찾는다는 것을 보여준다.

GAN은 다음과 같은 minimax problem을 풀고자 한다.

<img src="./Images/2.png" width=600 />

## 1. Global Optimality of $P_g={p}_{data}$

<img src="./Images/5.png" width=600 />

$\frac { a }{ a+b } $ 부분은 단순히 $y$에 대하여 미분을 하여 계산하면 된다. 여기서 주의할 점은 $log(.)$ 안의 값이 0이 될 수는 없으므로 미분할 때 $y$=0 또는 1인 경우는 따로 빼서 생각해주어야 한다. <br />
두 번째 문장의 $Supp({ p }_{ data })\cup Supp({ p }_{ g })$는 $V(G,D)$ 전개에서 두 번째 등호로 넘어가며 수식이 모두 $\int _{ x }^{  }{ \bullet dx }$ 안으로 들어갈 때 $p_z(z)\rightarrow p_g(x)$로 바뀌는 부분을 짚은 것이다. <br />
즉, 개념적으로 생각해보면 결국, ${ E }_{ z\sim { p }_{ z }(z) }(log(1-D(G(z)))={ E }_{ x\sim { p }_{ g }(x) }(log(1-D(x))$으로 기대값이 같다고 할 수 있기 때문에 등호가 성립하면서 $\int _{ x }^{  }{ \bullet dx }$로 범위가 $x$로 한정지어도 된다는 것의 당위성을 언급한 것이다.

결국 $min_g max_d V(G,D)$의 안 쪽의 max 부터 풀어주면 문제가 다음과 같이 reformulate 된다.

<img src="./Images/6.png" width=600 />

이를 이용하여 main theorem을 증명해보자.

<img src="./Images/7.png" width=600 />

Proof, For $p_g={p}_{data}$, ${D}^{*}_{G}(x)$=0.5 이고 따라서 다음 수식을 얻는다.

<img src="./Images/8.png" width=600 />

이 값이 Best possible value of $C(G)$라는 것을 알기 위해서 $C(G)$를 다음과 같이 표현해보자.

<img src="./Images/9.png" width=600 />

여기서 $KL$은 kullback-Leibler divergence이고 $JSD$는 Jensen-Shannon divergence이다. <br />
$JSD$는 항상 양수이고 두 distribution이 일치할 때만 0이므로 $C^*=-log(4)$가 $C(G)$의 global minimum이며 그 유일한 해가 $p_g={p}_{data}$임을 알 수 있다.

#### Cf) KL이란?

<img src="./Images/10.png" width=400 />

$P$라는 distribution이 있을 때, estimate 한 Q가 P와 얼마나 다른지 측정하는 값이다. 

즉, 위의 식은 KL의 성질을 이용하여,

<img src="./Images/11.png" width=600 />

위와 같이 나타내었다.

이렇게 $log(4)$를 더하고 빼준 후 $C(G)+log(4)$를 보면

<img src="./Images/12.png" width=600 />

이렇게 각 Expectation 안으로 $log(2)$를 각각 넣어주면 두 번쨰 등호의 수식에서 나오는 KL의 형태가 되는 것을 알 수 있다.

#### Cf) JSD 란?

<img src="./Images/13.png" width=600 />

여기서 $M=\frac { 1 }{ 2 } (P+Q)$이므로 그대로 원래의 수식에 비교 대조해보면, 마지막 세 번째 등호의 수식으로 넘어가는 것을 이해할 수 있다.

이제 앞서 정의한 minimax problem을 잘 풀면(즉, global optimal을 찾으면), generator가 만드는 probability distribution($p_g$)에서 data distribution(${p}_{data}$)과 정확히 일치하도록 할 수 있다는 것을 알았다. <br />
결국, Generator로 부터 뽑은 sample을 Discriminator가 실제와 구별할 수 없게 되는 것이다.

## Convergence of Algorithm 1

이 논문에서는 신경망 모델(MLP)을 사용하여 $G$와 $D$를 정의하고 각각을 fix한 상태에서 번갈아가며 문제를 풀어주는 전략을 제시하였다. <br />
이제 제시한 알고리즘이 문제를 잘 풀어주는가? 혹은 Global Optimum인 $p_g={p}_{data}$로 수렴하는가?를 확인해보자

<img src="./Images/17.png" width=600 />

<img src="./Images/14.png" width=600 />

증명에서 Convex function의 한 성질을 이해해보자.

"The subderivatives of a supremum of convex functions include the derivative of the function at the point where the maximum is attained."

이 문장을 상황에 맞게 변수를 넣고 수직으로 바꾸어 적으면 다음과 같다.

<img src="./Images/15.png" width=600 />

여기서 ${f}_{D}(p_g)$를 $U(p_g,D)$라 생각해보자.

위 증명에서 언급한 대로 $U(p_g,D)$는 $p_g$에 대하여 convex함수이다. <br />
그리고 이 함수의 supremum의 subderivatives라 함은 $\partial ({ sup }_{ D }U({ p }_{ g },D))$이며 단순하게 $\partial f$로 나타낼 수 있다. <br />
Subderivatives는 집합이므로 $\partial { f }_{ { D }^{ * } }({ p }_{ g })\in \partial f$임은 자명하다. <br />
여기서 앞서 증명한 Thm1에 의해 "${ sup }_{ D }U({ p }_{ g },D)$is convex in $p_g$ with a unique global optima"이므로 $p_g$에 대한 적은 업데이트만으로도 $p_{ g }\rightarrow { p }_{ data }$한다는 것을 보일 수 있다.

이제 논문에서 제시한 알고리즘이 실제로 global optimal을 찾아준다는 것까지도 증명하였다.

정리하면,

<img src="./Images/16.png" width=600 />

즉, MLP를 model로 사용하면 $G$가 parameter space에서 multiple critical points를 가지기 때문에 완전히 증명한 바와 합치하지 않는다는 것이다. <br />
다만 성능이 잘 나오는 것으로 미루어 보다, MLP가 이론적 보장이 부족할지라도 실용적으로 쓰이기에는 좋은 모델이라 할 수 있다.